In [ ]:
%matplotlib inline
import os
import copy
import subprocess
import matplotlib.pylab as plt
import numpy as np
from os.path import exists

from lib_mogi import *  # includes key functions

# GEOS 627 Inverse Problems and Parameter Estimation, Carl Tape
# GEOS 657 Microwave Remote Sensing, Franz Meyer
# University of Alaska Fairbanks
#
# Modeling subsurface volcanic sources using InSAR data
#
# contributors: Zhong Lu, Franz Meyer, Carl Tape, Amanda McPherson*

In [ ]:
# Define variables - InSAR
sample = 1100
line = 980
posting = 40.0
half_wave = 28.3

In [ ]:
#raise SystemExit("stop here")

In [ ]:
# Download and read in data - InSAR
displacement_map_path = 's3://asf-jupyter-data/E451_20000818_20020719.unw'
displacement_map = os.path.basename(displacement_map_path)
if not exists('./E451_20000818_20020719.unw'):
    !aws --region=us-east-1 --no-sign-request s3 cp $displacement_map_path $displacement_map

with open (displacement_map, 'rb') as f:    
    coh = np.fromfile(f, dtype='>f', count=-1)
    
observed_displacement_map = np.reshape(coh, (line, sample))

# Scale the measured and unwrapped InSAR phase into surface displacement in cm units and replace all nans with 0
observed_displacement_map = observed_displacement_map*half_wave/2.0/np.pi
where_are_NaNs = np.isnan(observed_displacement_map)
isokay = np.isreal(observed_displacement_map)
observed_displacement_map[where_are_NaNs] = 0

# Create a masked that removes invalid samples (low coherence) from the displacement map
observed_displacement_map_m = np.ma.masked_where(observed_displacement_map==0, observed_displacement_map)

print('data files is %i x %i' % (observed_displacement_map_m.shape))
print('line = %i' % line)
print('sample = %i' % sample)

In [ ]:
xvec,yvec = plot_model(observed_displacement_map_m, line, sample, posting, dpi=200)

In [ ]:
# function to create simulated displacement data based on Mogi source model parameters
# note: this function requires variables sample, line, posting
def mogi2insar(x, y, z, volume, iplot, imask):
    # Organizing model parameters
    bvc = [x, y, z, volume, 0, 0, 0, 0]
    bvc = np.asarray(bvc, dtype=object)
    bvc = np.transpose(bvc)
    
    # Setting acquisition parameters
    track = -13.3*np.pi / 180.0
    look  =  23.0*np.pi / 180.0
    plook = [-np.sin(look)*np.cos(track), np.sin(look)*np.sin(track), np.cos(look)]
    
    # Defining easting and northing vectors
    northing = np.arange(0, (line)*posting, posting) / 1000
    easting = np.arange(0, (sample)*posting, posting) / 1000
    northing_mat = np.tile(northing, (sample, 1))
    easting_mat = np.transpose(np.tile(easting, (line, 1)))
    northing_vec = np.reshape(northing_mat, (line*sample, 1))
    easting_vec = np.reshape(easting_mat, (line*sample, 1))
    
    # Handing coordinates and model parameters over to the rngchg_mogi function
    calc_range = rngchg_mogi(bvc[1], bvc[0], bvc[2], bvc[3], northing_vec, easting_vec, plook)
    
    # Reshaping surface displacement data derived via calc_forward_model_mogi()
    surface_displacement = np.reshape(calc_range, (sample,line))
    
    # return rotated surface displacement
    return np.transpose(np.fliplr(surface_displacement))